# COPASI Integration: Export & Import

Use HappyGene with COPASI via SBML Level 3 v2 format.

## What You'll Learn
- How to export configuration to SBML (for COPASI)
- How to import SBML files back to HappyGene
- How to verify round-trip fidelity
- COPASI workflow integration

## Setup: Create Configuration

In [ ]:
import tempfile
from pathlib import Path

from engine.domain.config import DamageProfile, HappyGeneConfig, KineticsConfig
from engine.io.sbml_export import export_to_sbml
from engine.io.sbml_import import import_from_sbml
from engine.io.sbml_validator import validate_sbml

# Create configuration
damage_profile = DamageProfile(
    dose_gy=3.0,
    population_size=1000
)

kinetics = KineticsConfig(
    recognition_rate=0.1,
    repair_rate=0.05,
    misrepair_rate=0.01,
    recovery_rate=0.02
)

config = HappyGeneConfig(
    damage_profile=damage_profile,
    kinetics=kinetics
)

print("✓ Configuration created")
print(f"  Dose: {config.damage_profile.dose_gy} Gy")
print(f"  Recognition rate: {config.kinetics.recognition_rate}")

## Export to SBML

Create SBML Level 3 v2 file that can be opened in COPASI.

In [ ]:
# Export to SBML
tmpdir = Path(tempfile.gettempdir())
sbml_path = tmpdir / "happygene_model.xml"

export_to_sbml(config, sbml_path)

print(f"✓ Exported to SBML: {sbml_path}")
print(f"  File size: {sbml_path.stat().st_size / 1024:.1f} KB")
print("  Format: SBML Level 3 Version 2")
print("  Compatible with: COPASI, libSBML, Systems Biology tools")

## Inspect SBML File

Preview the exported SBML structure.

In [ ]:
# Read and display SBML header
with open(sbml_path, 'r') as f:
    lines = f.readlines()

print("SBML File Structure:")
print("-" * 60)
for line in lines[:10]:
    print(line.rstrip())
print("...")
print(f"Total lines: {len(lines)}")

## Validate SBML

Check that the SBML file is valid.

In [ ]:
# Validate SBML
is_valid, errors = validate_sbml(sbml_path)

print(f"SBML Validation: {'✓ PASS' if is_valid else '✗ FAIL'}")

if errors:
    print(f"Errors: {len(errors)}")
    for error in errors[:5]:
        print(f"  - {error}")
else:
    print("No validation errors")

## Import from SBML

Load the SBML file back to recreate the configuration.

In [ ]:
# Import from SBML
recovered_config = import_from_sbml(sbml_path)

print("✓ Imported from SBML")
print(f"  Dose: {recovered_config.damage_profile.dose_gy} Gy")
print(f"  Population: {recovered_config.damage_profile.population_size}")
print(f"  Recognition rate: {recovered_config.kinetics.recognition_rate}")

## Verify Round-Trip Fidelity

Check that export→import preserves all data.

In [ ]:
# Compare original vs. recovered
print("Round-Trip Verification:")
print("-" * 60)

checks = [
    ("Dose (Gy)",
     config.damage_profile.dose_gy,
     recovered_config.damage_profile.dose_gy),
    ("Population",
     config.damage_profile.population_size,
     recovered_config.damage_profile.population_size),
    ("Recognition rate",
     config.kinetics.recognition_rate,
     recovered_config.kinetics.recognition_rate),
    ("Repair rate",
     config.kinetics.repair_rate,
     recovered_config.kinetics.repair_rate),
    ("Misrepair rate",
     config.kinetics.misrepair_rate,
     recovered_config.kinetics.misrepair_rate),
    ("Recovery rate",
     config.kinetics.recovery_rate,
     recovered_config.kinetics.recovery_rate),
]

all_match = True
for name, original, recovered in checks:
    match = abs(original - recovered) < 1e-10
    status = "✓" if match else "✗"
    print(f"{status} {name:<20} {original} → {recovered}")
    all_match = all_match and match

print("-" * 60)
print(f"Overall: {'✓ PERFECT FIDELITY' if all_match else '✗ DIFFERENCES DETECTED'}")

## COPASI Workflow

### Step 1: Export from HappyGene (Done Above)
Generated: `happygene_model.xml`

### Step 2: Import in COPASI
1. Open COPASI
2. File → Import → SBML
3. Select `happygene_model.xml`
4. Run COPASI simulations

### Step 3: Export from COPASI
1. In COPASI: File → Export → SBML
2. Save as `copasi_output.xml`

### Step 4: Import Back to HappyGene
See code below

In [ ]:
# Example: Import COPASI output
# copasi_output_path = Path("/path/to/copasi_output.xml")
# copasi_config = import_from_sbml(copasi_output_path)
#
# # Run HappyGene with COPASI-derived config
# from engine.simulator.batch import BatchSimulator
# batch = BatchSimulator(copasi_config)
# results = batch.run_batch(num_runs=100)

print("✓ COPASI workflow template ready")
print(f"  Export file: {sbml_path}")
print("  Ready for COPASI import/export cycle")

## Use Case: Compare Solvers

Run simulations with HappyGene (Python/SciPy ODE) vs. COPASI (commercial solver) and compare results.

In [ ]:
from engine.simulator.batch import BatchSimulator

# Run with HappyGene
batch = BatchSimulator(config)
hg_results = batch.run_batch(num_runs=50)
hg_stats = BatchSimulator.compute_statistics(hg_results)

print("HappyGene Results:")
print(f"  Mean repairs: {hg_stats['mean_repair_count']:.1f}")
print(f"  Mean time:    {hg_stats['mean_repair_time']:.4f}s")

print("\n→ Compare with COPASI results after running the exported model")
print("  Differences should be <0.1% if solvers are equivalent")

## Next Steps

1. **Parameter Sensitivity**: See notebook 05 for systematic parameter variation
2. **Publication**: Export SBML as supplementary material for reproducibility
3. **Tools Integration**: Use with other SBML-compatible tools (libSBML, etc.)